https://github.com/tesseract-ocr/tesseract/tree/4.1

https://github.com/tesseract-ocr/tessdata_best

https://github.com/tesseract-ocr/langdata_lstm

In [1]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (19.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [2]:
!mkdir fonts output train

In [15]:
font_name = 'Times New Roman Cyr'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
#permission of tesseract folder
!chmod 755 -R /content/drive/MyDrive/tesseract/src/training/tesstrain.sh

In [17]:
#'rus.lstm' generating from 'rus.traineddata' for further fine tuning
!combine_tessdata -e /content/drive/MyDrive/tesseract/tessdata/rus.traineddata rus.lstm

Extracting tessdata components from /content/drive/MyDrive/tesseract/tessdata/rus.traineddata
Wrote rus.lstm
Version string:4.00.00alpha:rus:synth20170629:[1,36,0,1Ct3,3,16Mp3,3Lfys64Lfx96Lrx96Lfx512O1c1]
17:lstm:size=11742451, offset=192
18:lstm-punc-dawg:size=5154, offset=11742643
19:lstm-word-dawg:size=3543106, offset=11747797
20:lstm-number-dawg:size=2066, offset=15290903
21:lstm-unicharset:size=7582, offset=15292969
22:lstm-recoder:size=1129, offset=15300551
23:version:size=80, offset=15301680


In [18]:
#train data
!rm -rf train/*
! /content/drive/MyDrive/tesseract/src/training/tesstrain.sh --fonts_dir fonts \
	--fontlist '{font_name}' \
	--lang rus \
	--linedata_only \
	--langdata_dir /content/drive/MyDrive/langdata_lstm \
	--tessdata_dir /content/drive/MyDrive/tesseract/tessdata \
	--save_box_tiff \
	--maxpages 10 \
	--output_dir train


=== Starting training for language 'rus'
[Wed Apr 20 12:39:16 UTC 2022] /usr/bin/text2image --fonts_dir=fonts --ptsize 12 --font=Times New Roman Cyr --outputbase=/tmp/font_tmp.nS90yP695x/sample_text.txt --text=/tmp/font_tmp.nS90yP695x/sample_text.txt --fontconfig_tmpdir=/tmp/font_tmp.nS90yP695x
Rendered page 0 to file /tmp/font_tmp.nS90yP695x/sample_text.txt.tif

=== Phase I: Generating training images ===
Rendering using Times New Roman Cyr
[Wed Apr 20 12:39:18 UTC 2022] /usr/bin/text2image --fontconfig_tmpdir=/tmp/font_tmp.nS90yP695x --fonts_dir=fonts --strip_unrenderable_words --leading=32 --xsize=3600 --char_spacing=0.0 --exposure=0 --outputbase=/tmp/rus-2022-04-20.54h/rus.Times_New_Roman_Cyr.exp0 --max_pages=10 --font=Times New Roman Cyr --ptsize 12 --text=/content/drive/MyDrive/langdata_lstm/rus/rus.training_text
Rendered page 0 to file /tmp/rus-2022-04-20.54h/rus.Times_New_Roman_Cyr.exp0.tif
Rendered page 1 to file /tmp/rus-2022-04-20.54h/rus.Times_New_Roman_Cyr.exp0.tif
Render

In [19]:
%%shell
#finetune
rm -rf output/* 
OMP_THREAD_LIMIT=16 lstmtraining \
	--continue_from rus.lstm \
	--model_output output/font_name \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/rus.traineddata \
	--train_listfile train/rus.training_files.txt \
	--max_iterations 200

Loaded file rus.lstm, unpacking...
Continuing from rus.lstm
Loaded 520/520 pages (1-520) of document train/rus.Times_New_Roman_Cyr.exp0.lstmf
2 Percent improvement time=8, best error was 100 @ 0
At iteration 8/100/100, Mean rms=0.191%, delta=0.148%, char train=0.352%, word train=1.917%, skip ratio=0%,  New best char error = 0.352 Transitioned to stage 1 wrote best model:output/font_name0.352_8.checkpoint wrote checkpoint.

2 Percent improvement time=9, best error was 100 @ 0
At iteration 9/200/200, Mean rms=0.158%, delta=0.082%, char train=0.201%, word train=1.058%, skip ratio=0%,  New best char error = 0.201 wrote best model:output/font_name0.201_9.checkpoint wrote checkpoint.

Finished! Error rate = 0.201


In [20]:
%%shell
#output
lstmtraining --stop_training \
	--continue_from output/font_name_checkpoint \
	--traineddata /content/drive/MyDrive/tesseract/tessdata/rus.traineddata \
	--model_output output/font_name.traineddata

Loaded file output/font_name_checkpoint, unpacking...


example

*tesseract.exe image.jpg text.txt --psm 12 -l fontname*